In [1]:
# installs necessary packages 
# remove --quiet to see the verbose output


%pip install -U --quiet langchain-core langchain-community==0.2.6 youtube_search Wikipedia grandalf langchain-ollama



Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''You are a movie recommendation system. 
    Recommend a movie based on the genre and actor below.
    Genre: {genre}
    Actor: {actor}
    
    Output the movie name in the form of a json e.g. {{"movie_name": "movie name"}}. 
    Output only the json without any additional information or side notes.
    ''')

# test
prompt_template.format(genre="drama", actor="Brad Pitt")

'You are a movie recommendation system. \n    Recommend a movie based on the genre and actor below.\n    Genre: drama\n    Actor: Brad Pitt\n    \n    Output the movie name in the form of a json e.g. {"movie_name": "movie name"}. \n    Output only the json without any additional information or side notes.\n    '

In [3]:
from langchain_ollama import ChatOllama 
import json

# function to invoke the llm 
def invoke_llm(prompt):
    print(f"Invoking LLM with prompt: {prompt}")
    llm = ChatOllama(
        model="llama3.1:8b",
        temperature=1,
        max_tokens=512
    )
    ai_message = llm.invoke(prompt)
    return ai_message.content

# function to extract the movie name from the json output of the llm
def extract_movie (text:str) -> str:
    return json.loads(text)['movie_name']

# test LLM
invoke_llm(prompt_template.format(genre="Action", actor="Al Pacino"))


Invoking LLM with prompt: You are a movie recommendation system. 
    Recommend a movie based on the genre and actor below.
    Genre: Action
    Actor: Al Pacino
    
    Output the movie name in the form of a json e.g. {"movie_name": "movie name"}. 
    Output only the json without any additional information or side notes.
    


'{"movie_name": "Carlito\'s Way"}'

In [4]:
# test extract movie fumction
movie = extract_movie(invoke_llm(prompt_template.format(genre="Action", actor="Al Pacino")))
movie

Invoking LLM with prompt: You are a movie recommendation system. 
    Recommend a movie based on the genre and actor below.
    Genre: Action
    Actor: Al Pacino
    
    Output the movie name in the form of a json e.g. {"movie_name": "movie name"}. 
    Output only the json without any additional information or side notes.
    


'Serpico'

In [4]:
from langchain_community.retrievers import WikipediaRetriever

# function to retrieve movie info from wikipedia
def retrieve_movie_info(movie):
  print(f"Searching for {movie} movie info on wikipedia")
  retriever = WikipediaRetriever()
  docs = retriever.invoke(input=f"Hollywood english movie: {movie}")
  return docs[0].metadata['summary']

# test retrieve movie info function
movie_plot = retrieve_movie_info("Collateral Damage")
print(movie_plot)

Searching for Collateral Damage movie info on wikipedia
Collateral Damage is a 2002 American vigilante action-thriller film directed by Andrew Davis and starring Arnold Schwarzenegger, Elias Koteas, Francesca Neri, Cliff Curtis, John Leguizamo, and John Turturro. The film tells the story of Los Angeles firefighter Gordon Brewer (Schwarzenegger), who seeks to avenge his son's and wife's deaths at the hands of a guerrilla commando, by traveling to Colombia and facing his family's killers. 
Collateral Damage was released in the United States on February 8, 2002, to negative reviews, and was a commercial failure.


In [5]:
from langchain_community.tools import YouTubeSearchTool

# function to get the youtube trailer link
def get_youtube_trailer(movie):
  print(f"Searching for {movie} movie trailer on youtube")
  tool = YouTubeSearchTool()
  return tool.run(f"{movie} movie trailer").split("'")[1]


#test get youtube trailer function
get_youtube_trailer("The Shawshank Redemption")

Searching for The Shawshank Redemption movie trailer on youtube


'https://www.youtube.com/watch?v=PLl99DlL6b4&pp=ygUmVGhlIFNoYXdzaGFuayBSZWRlbXB0aW9uIG1vdmllIHRyYWlsZXI%3D'

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# create a chain to ask LLM for a movie recommendation and parse the output
recommend_movie_chain = (
                  prompt_template | 
                  invoke_llm | 
                  StrOutputParser() | 
                  extract_movie )

# test chain
#recommend_movie_chain.invoke(input={"genre":"action", "actor":"al pacino"})

### result
# 'House of Gucci'


wiki_template = PromptTemplate.from_template(
    """ 
    Summarize the below movie description in a few sentences, only output the summary nothing else:: 
    
    {wiki_dump}
    
    """
)

# chain for generating movie plot from wikipedia
movie_plot_chain = (
                retrieve_movie_info 
                | wiki_template 
                | invoke_llm 
                | StrOutputParser())

# chain for grabbing movie trailer from youtube
youtube_trailer_chain = get_youtube_trailer

final_response = PromptTemplate.from_template(
  """ 
    Combine the results from movie name, movie plot description and youtube trailer link below into  a well written movie recommendation::
    {movie}
    {movie_plot}
    {youtube_trailer}
    """
)

# chain of chains
chain = (
          recommend_movie_chain | StrOutputParser()
          | {"movie": RunnablePassthrough(), "movie_plot": movie_plot_chain , "youtube_trailer": youtube_trailer_chain} 
          | final_response 
          | invoke_llm       
        )

print(chain.invoke(input={"genre":"action", "actor":"angelina jolie"}))

Invoking LLM with prompt: text='You are a movie recommendation system. \n    Recommend a movie based on the genre and actor below.\n    Genre: action\n    Actor: angelina jolie\n    \n    Output the movie name in the form of a json e.g. {"movie_name": "movie name"}. \n    Output only the json without any additional information or side notes.\n    '
Searching for Tomb Raider movie info on wikipediaSearching for Tomb Raider movie trailer on youtube

Invoking LLM with prompt: text=' \n    Summarize the below movie description in a few sentences, only output the summary nothing else:: \n    \n    Tomb Raider is a 2018 action-adventure film directed by Roar Uthaug, with a screenplay by Geneva Robertson-Dworet and Alastair Siddons, from a story by Evan Daugherty and Robertson-Dworet. An American and British co-production, it is based on the 2013 video game of the same name, with some elements of its sequel by Crystal Dynamics, a reboot, and the third installment in the Tomb Raider film serie

In [7]:
chain.get_graph().print_ascii()

                                      +-------------+                                        
                                      | PromptInput |                                        
                                      +-------------+                                        
                                             *                                               
                                             *                                               
                                             *                                               
                                    +----------------+                                       
                                    | PromptTemplate |                                       
                                    +----------------+                                       
                                             *                                               
                                             *              